In [2]:
!free -h 
!ls -alh decompressed

# Source:
# https://www.kaggle.com/Cornell-University/arxiv
# http://arxiv.org/abs/1905.00075v1

              total        used        free      shared  buff/cache   available
Mem:           125G         12G         93G         17M         19G        111G
Swap:            0B          0B          0B
total 3.7G
drwxrwxr-x 3 xiaowentao xiaowentao  227 11月 21 23:39 .
drwxrwxr-x 4 xiaowentao xiaowentao  245 11月 22 00:41 ..
-rw-rw-r-- 1 xiaowentao xiaowentao 692M 9月  22 23:57 arxiv-metadata-hash-abstracts-v0.2.0-2019-03-01.json
-rw-rw-r-- 1 xiaowentao xiaowentao 2.7G 11月 15 00:54 arxiv-metadata-oai-snapshot.json
-rw-rw-r-- 1 xiaowentao xiaowentao 185M 9月  22 23:46 authors-parsed-v0.2.0-2019-03-01.json
-rw-rw-r-- 1 xiaowentao xiaowentao 126M 9月  22 23:46 internal-references-v0.2.0-2019-03-01.json
drwxrwxr-x 2 xiaowentao xiaowentao    6 9月  22 23:53 .ipynb_checkpoints


In [3]:
import json
from pathlib import Path
from time import time
from itertools import chain
import random
import socket
import socks
import numpy as np
import os
from queue import Queue
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'
# Force use CPU
# https://github.com/tensorflow/tensorflow/issues/31135
import tensorflow as tf
print(tf.__version__)
# Set CPU as available physical device
# physical_devices = tf.config.list_physical_devices('GPU')
# print(f'physical_devices: {physical_devices}')
# tf.config.set_visible_devices(physical_devices[0:2], 'GPU')
import tensorflow_hub as hub
socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 1080)
socket.socket = socks.socksocket

# params
verbose = True
min_edge = 20
max_edge = 400
min_graph = 500

metadata = []
st = time()
with open(list(Path('decompressed').glob('arxiv-metadata-oai*'))[0], 'r') as f:
    for line in f.readlines():
        metadata.append(json.loads(line))
    metadata_id_dict = {v['id']: i for i, v in enumerate(metadata)}
    if verbose:
        print(f'Done with {time() - st}s.')
        print('Metadata:\n', '\n\n'.join(map(lambda x: str(x), metadata[:1])))
        print(metadata[metadata_id_dict[citation['1307.7980'][2]]])

citation = []
st = time()
with open(list(Path('decompressed').glob('internal-references*'))[0], 'r') as f:
    citation = json.loads(f.readlines()[0])
    if verbose:
        print(f'Done with {time() - st}s\n',
              'Citation:\n', citation['1307.7980'])

Done with 67.24637126922607s.
Metadata:
 {'id': '0704.0001', 'submitter': 'Pavel Nadolsky', 'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan", 'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies', 'comments': '37 pages, 15 figures; published version', 'journal-ref': 'Phys.Rev.D76:013009,2007', 'doi': '10.1103/PhysRevD.76.013009', 'report-no': 'ANL-HEP-PR-07-12', 'categories': 'hep-ph', 'license': None, 'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\

In [80]:
print(f'metadata: {len(metadata)}, citation: {len(citation)}')
category_stat = {}
category_stat_coarse = {}
for p in citation:
    if p in metadata_id_dict:
        c = metadata[metadata_id_dict[p]]['categories']
        if ' ' not in c:
            # if single category
            if c not in category_stat:
                category_stat[c] = 1
            else:
                category_stat[c] += 1
            c = c.split('.')[0]
            if c not in category_stat_coarse:
                category_stat_coarse[c] = 1
            else:
                category_stat_coarse[c] += 1
if verbose:
    print(f'category_stat (len={len(category_stat)}): {category_stat}' +
          f'({len(list(filter(lambda x: x[1] >= 2000, category_stat.items())))}' +
          f' keys with values >= 2000)\n\ncategory_stat_coarse (len=' +
          f'{len(category_stat_coarse)}): {category_stat_coarse}')

category_target_stat = list(filter(lambda x: x[0].startswith("cs.") and x[1] >= 2000,
                                   category_stat.items()))
category_target = set(map(lambda x: x[0], category_target_stat))
print('category_stat of subclasses of "cs" with more than 2000 papers:\n' +
      f'{category_target_stat}\n')

metadata: 1793457, citation: 1354753
category_stat (len=149): {'gr-qc': 22341, 'q-bio.PE': 1490, 'q-bio.BM': 444, 'q-bio.MN': 475, 'q-bio.QM': 494, 'q-bio.SC': 122, 'q-bio.NC': 983, 'q-bio.GN': 284, 'q-bio.CB': 141, 'q-bio.TO': 146, 'q-bio.OT': 104, 'hep-ph': 68452, 'physics.soc-ph': 1583, 'physics.ins-det': 2211, 'physics.atom-ph': 4372, 'physics.class-ph': 1229, 'physics.chem-ph': 1672, 'physics.geo-ph': 527, 'physics.hist-ph': 358, 'physics.optics': 6227, 'physics.flu-dyn': 4727, 'physics.ed-ph': 618, 'physics.ao-ph': 520, 'physics.plasm-ph': 3384, 'physics.gen-ph': 3596, 'physics.atm-clus': 299, 'physics.acc-ph': 1551, 'physics.data-an': 630, 'physics.comp-ph': 963, 'physics.med-ph': 460, 'physics.pop-ph': 177, 'physics.bio-ph': 548, 'physics.space-ph': 295, 'math.GT': 5844, 'math.HO': 809, 'math.SP': 1173, 'math.DG': 10972, 'math.CV': 4107, 'math.CO': 16819, 'math.OA': 2620, 'math.AG': 13844, 'math.RA': 4045, 'math.GR': 5177, 'math.PR': 15722, 'math.GM': 1303, 'math.OC': 7492, 'ma

In [265]:
check_valid_id = lambda pid: pid in metadata_id_dict and pid in citation
dbg = {c: {'nodes': [], 'uni_nodes': [], 'edge': []} for c in category_target}
graphs = []
dataset = {c: [] for c in category_target}
cnt = 0
cnt_empty_ref = 0
cnt_fail = 0
st = time()
print('start construct graphs.')

for pid, refs in filter(lambda x: len(x[1]) >= min_width, citation.items()):
    cnt += 1
    if cnt % 40000 == 0:
        print(f'cnt: {cnt / 10000:.2f}w/135w, with {(time() - st) / cnt * 10000:.1f}s/10000')
    category = metadata[metadata_id_dict[pid]]['categories']
    if category not in category_target:
        continue
    refs = list(filter(check_valid_id, refs))
    if len(refs) == 0:
        cnt_empty_ref += 1
        continue
    q = Queue()
    for r in refs:
        q.put(r)
    # one paper with its ciatations construct one edge
    edges = [[pid, ] + refs, ]
    while q.qsize() > 0 and len(edges) <= max_edge:
        curr = q.get()
        refs = list(filter(check_valid_id, citation[curr]))
        edges.append([curr, ] + refs)
        for r in refs:
            q.put(r)
    if len(edges) >= min_edge:
        graphs.append([pid, edges])
        dataset[category].append(len(graphs) - 1)
        nodes = list(chain.from_iterable(edges))
        dbg[category]['nodes'].append(len(nodes))
        dbg[category]['uni_nodes'].append(len(set(nodes)))
        dbg[category]['edge'].append(len(edges))
    else:
        cnt_fail += 1

dataset_stat = {k: len(v) for k, v in dataset.items()}
dbg = {c: dict(map(lambda x: [x[0], round(sum(x[1]) / len(x[1]), 2)], dbg[c].items())) for c in dbg}
if verbose:
    print(f'cnt_fail = {cnt_fail}, cnt_empty_ref = {cnt_empty_ref}')
print(f'#graph = {len(graphs)}\n\ndataset: {dataset_stat}\n\n' +
      f'{dbg}\n')

start
cnt: 4.00w/135w, with 0.0s/10000
cnt: 8.00w/135w, with 0.0s/10000
cnt: 12.00w/135w, with 0.0s/10000
cnt: 16.00w/135w, with 0.1s/10000
cnt: 20.00w/135w, with 0.4s/10000
cnt: 24.00w/135w, with 0.8s/10000
cnt: 28.00w/135w, with 1.0s/10000
cnt: 32.00w/135w, with 1.5s/10000
cnt: 36.00w/135w, with 1.6s/10000
cnt: 40.00w/135w, with 1.6s/10000
cnt: 44.00w/135w, with 1.6s/10000
cnt: 48.00w/135w, with 1.7s/10000
cnt: 52.00w/135w, with 1.7s/10000
cnt: 56.00w/135w, with 1.7s/10000
cnt: 60.00w/135w, with 1.8s/10000
cnt: 64.00w/135w, with 1.7s/10000
cnt: 68.00w/135w, with 1.7s/10000
cnt: 72.00w/135w, with 1.6s/10000
debug: cnt_fail = 6369, cnt_empty_ref = 529
#graph = 12982

dataset: {'cs.DC': 244, 'cs.AI': 553, 'cs.LG': 885, 'cs.CR': 273, 'cs.CL': 2318, 'cs.CV': 7435, 'cs.DS': 621, 'cs.NI': 283, 'cs.SY': 166, 'cs.LO': 204}

{'cs.DC': {'nodes': 2202.905737704918, 'uni_nodes': 455.3524590163934, 'edge': 384.5163934426229}, 'cs.AI': {'nodes': 2280.4068716094034, 'uni_nodes': 420.9186256781193, '

In [266]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


def get_paper_text(pid):
    meta = metadata[metadata_id_dict[pid]]
    return str(meta['title'].replace('\n', ' ') + '. ' + meta['abstract'].replace('\n', ' '))


dataset_target_final = set([k for k, v in dataset_stat.items() if v >= min_graph])
print(f'dataset_target_final: {dataset_target_final}')
# if #samples >= 1000, then subsampling to 700
dataset_downsample = {c: (random.sample(d, len(d)) if dataset_stat[c] <= 1000 else \
           random.sample(d, 700)) for c, d in dataset.items() if c in dataset_target_final}
print(f'dataset: {[[c, len(v)] for c, v in dataset_downsample.items()]}')

st = time()
dataset_pids = set([graphs[i][0] for i in list(chain.from_iterable(
    dataset_downsample.values()))])
# Get node features
pids = list(set(chain.from_iterable([set(chain.from_iterable(x[1])) for x in graphs if x[0] in dataset_pids])))
# decrease memory usage
graphs = [g if g[0] in dataset_pids else [g[0], None] for g in graphs]
corpus = {p: get_paper_text(p) for p in pids}
print(f'#corpus={len(corpus)}')
batch_size = 2048
for idx in range(int(np.ceil(len(corpus) / batch_size))):
    curr_range = pids[idx * batch_size: (idx + 1) * batch_size]
    # => 512-d feature vector
    emb = embed([corpus[i] for i in curr_range]).numpy()
    for i, pid in enumerate(curr_range):
        corpus[pid] = emb[i]
print(f'Embedding {len(corpus)} paper corpus with {time() - st}s.')

2.2.0
dataset_target_final: {'cs.AI', 'cs.LG', 'cs.CL', 'cs.CV', 'cs.DS'}
#corpus=67809
Embedding 67809 paper corpus with 46.120678186416626s.


In [ ]:
# 导出文件：graphs: [..., [pid_i, edges_i], ...], 其中 pid_i 为第 i 个 graph 构造的时候的起始论文的 ArXiv ID，
# edges_i 为  [..., [pid_j0, ..., pid_jn], ...] 为第 i 个 graph 的超边集，每一条边长度不一样，例如第 j 条超边
# 长度为 jn，pid_j0 表示论文的 ArXiv ID。
# dataset: {class: [gid_0, ...], ...}，共有五个类：{'cs.AI', 'cs.LG', 'cs.CL', 'cs.CV', 'cs.DS'},
# 每个类对应一个数组，数组里面就是相应的 graph 的索引。
# corpus: {pid: paper_embd, ...}, 为 ArXiv ID 对应 paper 的标题和摘要的 embedding（512-D vector）。
st = time()
np.savez_compressed('arxiv_hypergraph_preprocessed.npz',
                    graphs=graphs,
                    dataset=dataset_downsample,
                    corpus=corpus)
print(f'File written into {os.path.abspath("arxiv_hypergraph_unsplited.npz")}' +
      f'\n(size={os.path.getsize("arxiv_hypergraph_unsplited.npz")})' +
      f' with {time() - st}s.')

In [267]:
def construct_hypergraph(edges, test_pid, test=False):
    # construct hypergraph H with shape (N, E)
    pid2nid = {pid: i for i, pid in enumerate(chain.from_iterable(edges))}
    coo_matrix_H = list(chain.from_iterable([
        [[pid2nid[p], e_id] for p in edge if (test or (p not in test_pid))] for e_id, edge in enumerate(edges)]))
    coo_matrix_H = sorted(coo_matrix_H, key=lambda x: x[0])
    row = [i[0] for i in coo_matrix_H]
    col = [i[1] for i in coo_matrix_H]
    # currently all elements in H are 1
    data = [1.] * len(row)
    node_features = np.stack([corpus[i[0]] for i in sorted(
        [[k, v] for k, v in pid2nid.items()], key=lambda x: x[1])])
    return {'H': [data, (row, col)], 'X': node_features}

st = time()
# split: 70% training, 30% testing (10% validation, 20% testing)
ratio = .7
dataset_train = {c: d[:int(ratio * len(d))] for c, d in dataset_downsample.items()}
dataset_test = {c: d[int(ratio * len(d)):] for c, d in dataset_downsample.items()}
# In order to prevent a leaking of the test set into the trainingset,
# using the train/test partition that we omitted citations from articles
# in the training set which connect to the test set, but retained citations
# in the test set which connect to the training set.
test_pid_by_class = {c: set([graphs[i][0] for i in d]) for c, d in dataset_test.items()}
dataset_train = {c: {graphs[i][0]: construct_hypergraph(
    graphs[i][1], test_pid_by_class[c]) for i in d} for c, d in dataset_train.items()}
dataset_test = {c: {graphs[i][0]: construct_hypergraph(
    graphs[i][1], test_pid_by_class[c], test=True) for i in d} for c, d in dataset_test.items()}
print(f'Construct hypergraph done with {time() - st}s.')

Construct hypergraph done with 23.48652410507202s.


In [1]:
!uptime && free -h && nvidia-smi

 21:51:04 up 11 days,  4:22, 21 users,  load average: 47.64, 46.37, 46.11
              total        used        free      shared  buff/cache   available
Mem:           125G         34G         56G         83M         34G         90G
Swap:            0B          0B          0B
Tue Nov 24 21:51:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:18:00.0 Off |                  N/A |
| 22%   34C    P2    62W / 250W |   5508MiB / 1101